In [1]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack, Simulation
from flowermd.library import OPLS_AA_PPS,PPS
from flowermd.utils import get_target_box_mass_density
import unyt as u
import hoomd

cpu = hoomd.device.CPU()

In [2]:
pps_chains = PPS(lengths=[15,17,19,22,30], num_mols=[3,2,1,3,1])
system = Pack(molecules=[pps_chains], density=0.1)
system.apply_forcefield(
    r_cut=2.5,
    force_field=OPLS_AA_PPS(),
    auto_scale=True,
    remove_hydrogens=True)

In [4]:
gsd_path=('ua-pps-trajectory0.gsd')
log_path=('ua-pps-log0.txt')
sim = Simulation.from_system(
    system=system,
    gsd_write_freq=int(1e5),
    gsd_file_name=gsd_path,
    log_file_name=log_path,
    dt=0.001,
    log_write_freq=int(1e5),
)
#target_box = get_target_box_mass_density(density=system.density, mass=system.mass.to("g"))
#lj_sim.run_update_volume(final_box_lengths=target_box,kT=5.0,n_steps=1e4,tau_kt=5*lj_sim.dt,period=10)
sim.run_NVT(n_steps=int(1e7), kT=5.0, tau_kt=10*sim.dt)
sim.flush_writers()
sim.save_restart_gsd("pps_restart.gsd")

No charged group detected, skipping electrostatics.


'\ngsd_path=(\'pps_trajectory0.gsd\')\nlog_path=(\'pps_log0.txt\')\nsystem.to_gsd("pps_init_frame.gsd")\nlj_sim = Simulation.from_system(\n    system=system,\n    gsd_write_freq=int(1e5),\n    gsd_file_name=gsd_path,\n    log_file_name=log_path,\n    dt=0.001,\n    log_write_freq=int(1e5),\n)\ntarget_box = get_target_box_mass_density(density=system.density, mass=system.mass.to("g"))\nlj_sim.run_update_volume(final_box_lengths=target_box,kT=5.0,n_steps=1e4,tau_kt=5*lj_sim.dt,period=10)\nlj_sim.run_NVT(n_steps=int(1e7), kT=5.0, tau_kt=10*lj_sim.dt)\nlj_sim.flush_writers()\nlj_sim.save_restart_gsd("pps_restart.gsd")'

In [40]:
from cmeutils.sampling import is_equilibrated
shrink_cut = 0
log = np.genfromtxt('log1.txt', names=True)
pe = log["mdcomputeThermodynamicQuantitiespotential_energy"]
# equil: bool (True or False), t0: int (starting data point that you can sample from), neff: number of data points in equilibrated region
print(pe[shrink_cut:])
equil, t0, g, neff = is_equilibrated(data=pe[shrink_cut:], threshold_neff=100) 
if equil:
    print("True")
else:
    # If it's not equilibrated, job.doc.equilibrated stays false, and the "run-longer" operation in project.py will run next time we submit this job
    print("False")

[ 727.48708  834.95685  760.49311  931.42768  805.28198  739.72208
  842.53447  866.46243  891.76662  856.44867  893.39384  910.12994
  806.3869   796.34247  916.87258  992.37378  854.15857  890.50166
  694.50774  815.77832  770.93469  761.83423  878.11445  790.21439
  889.78855  825.43489  759.88784  875.4892   773.8543   787.80066
  891.3053   792.40433  779.47384  912.21186  893.30654 1138.96
  759.21959  688.68178  811.87708  801.37126  798.66241  810.76379
  851.67323  757.94149  728.54321  809.97761  677.94876  867.8237
  884.74994  871.43893  867.32038  829.08842  653.87137  951.5481
 1153.06     622.55822  939.93338  811.08511  874.12353  814.97686
  872.05211  850.60194  897.88662  876.42928  908.56003  809.85007
  786.21598  709.11911  781.78074  934.87801  939.56702  826.21437
  903.27135 1029.36     620.09085  914.41993  731.8364   720.76355
  770.03664  849.84074  822.11342  904.09387  912.58049  934.91835
  826.9228   838.05606  824.38168  834.77103  906.09295  887.32153
